Gather, process examples of text from similar domains as the white supremacist dataset that isn't white supremacist

# Reddit data (forum data)
Scraped by get_reddit.py

In [46]:
# Load data
import os
import pandas as pd

dirpath = '../tmp/'
fpaths = sorted([fname for fname in os.listdir(dirpath) if 'subreddit_comments' in fname])
dfs = []
for fname in fpaths:
    print(fname)
    fpath = os.path.join(dirpath, fname)
    dfs.append(pd.read_csv(fpath, index_col=0, engine='python').assign(year=fname[:4]))
    
data = pd.concat(dfs)
data['year'] = data.year.astype(int)
data

2011_politics_subreddit_comments.csv
2012_politics_subreddit_comments.csv
2013_politics_subreddit_comments.csv
2014_politics_subreddit_comments.csv
2015_politics_subreddit_comments.csv
2016_politics_subreddit_comments.csv


,author,body,created_utc,id,parent_id,score,created,text,year
48865,[deleted],"If I have 3 kids it's not likely I'd be missing out on much on weeknights it takes 1/2 hour to prepare meal. Plus cooking can be a fun family activity if you make it so. Making huge amounts and freezing also cuts time in about half for most foods. You do have a point though, as 95% of the time I get fast food it's a quick/convenience thing.",1325043053,c3bpt8q,t1_c3bohzb,1,1.325057e+09,"If I have 3 kids it's not likely I'd be missing out on much on weeknights it takes 1/2 hour to prepare meal. Plus cooking can be a fun family activity if you make it so. Making huge amounts and freezing also cuts time in about half for most foods. You do have a point though, as 95% of the time I get fast food it's a quick/convenience thing.",2011
6229,zorbathustra,"What if individual executives here held more directly liable for the content of the speech they approve? This would seem to be the reason that journalists want a federal shield law, and why Judith Miller went to jail.\n\n\nCorporate liability shields individuals who authorize illegal and immoral acts.\n\n\nConsider the EU Microsoft Anti-Competition Case: Microsoft was fined $2 billion for operating in open violation of the law for over a decade. Because nobody went to jail, executives had no incentive to obey the law. Legal fees become simply another business expense, perhaps cheaper in the long run than marketing and advertising.\n\n\nWe saw something similar with the retroactive immunity for telecommunications carriers that spied on us citizens (where, incidentally, Verizon justified handing over customer data to the National Security Agency as protected free speech).",1325342043,c3cpubb,t1_c3coi5e,-2,1.325356e+09,"What if individual executives here held more directly liable for the content of the speech they approve? This would seem to be the reason that journalists want a federal shield law, and why Judith Miller went to jail.\n\n\nCorporate liability shields individuals who authorize illegal and immoral acts.\n\n\nConsider the EU Microsoft Anti-Competition Case: Microsoft was fined $2 billion for operating in open violation of the law for over a decade. Because nobody went to jail, executives had no incentive to obey the law. Legal fees become simply another business expense, perhaps cheaper in the long run than marketing and advertising.\n\n\nWe saw something similar with the retroactive immunity for telecommunications carriers that spied on us citizens (where, incidentally, Verizon justified handing over customer data to the National Security Agency as protected free speech).",2011
62876,burningpineapples,You could always go the Visual-Basic way\n&lt;&gt;,1324970410,c3bhbfi,t1_c3bdw69,1,1.324985e+09,You could always go the Visual-Basic way\n&lt;&gt;,2011
20181,sheasie,"That's not true. There is no impunity against torture. \n\nTorture is flat-out illegal, and it always will be. **America never has tortured, nor will America ever torture.** We are innovators. We ""enhance interrogate"" (to death, on more occasions than I know -- no doubt). \n\nSo, ""no"". Private Military Contractors, absolutely do not have impunity when it comes to torture. But it's a non-issue, because America does not torture. It's that simple. Get it through your thick skull, rock head.\n\n",1325237238,c3ce40j,t1_c3c9v1j,2,1.325252e+09,"That's not true. There is no impunity against torture. \n\nTorture is flat-out illegal, and it always will be. **America never has tortured, nor will America ever torture.** We are innovators. We ""enhance interrogate"" (to death, on more occasions than I know -- no doubt). \n\nSo, ""no"". Private Military Contractors, absolutely do not have impunity when it comes to torture. But it's a non-issue, because America does not torture. It's that simple. Get it through your thick skull, rock head.\n\n",2011
53348,[deleted],Wow a clever Romney quip! Now only if I were 30 years older I might have understo

In [7]:
# Compare with white supremacist dataset
# Load white supremacist dataset to count posts over time
import datetime

path = '../tmp/white_supremacist_corpus.pkl'
ws_data = pd.read_pickle(path)

# Select forum data, Group by year
yearly = ws_data.query('domain=="forum"').groupby(by=ws_data.timestamp.dt.year)['text'].count()
lookup = pd.DataFrame(yearly)
lookup['begin'] = pd.to_datetime(yearly.index.astype(int).astype(str), format='%Y')
lookup['end'] = lookup.begin.shift(-1, fill_value = datetime.datetime(2020,1,1))
lookup.index.name = 'year'
lookup.index = lookup.index.astype(int)
lookup.rename(columns={'text': 'post_count'}, inplace=True)
lookup

,post_count,begin,end
year,,,
2001,1439,2001-01-01,2002-01-01
2002,13302,2002-01-01,2003-01-01
2003,27470,2003-01-01,2004-01-01
2004,50783,2004-01-01,2005-01-01
2005,54377,2005-01-01,2006-01-01
2006,80554,2006-01-01,2007-01-01
2007,63544,2007-01-01,2008-01-01
2008,74203,2008-01-01,2009-01-01
2009,94895,2009-01-01,2010-01-01


In [47]:
# Tokenize, lowercase Reddit data
import nltk
from tqdm.notebook import tqdm

def preprocess(inp):
    return ' '.join(nltk.word_tokenize(str(inp))).lower()

data['processed'] = [preprocess(x) for x in tqdm(data.body)]
data['word_count'] = data.processed.str.split().str.len()
reddit_wc = data.groupby('year').word_count.sum()
reddit_wc

  0%|          | 0/461359 [00:00<?, ?it/s]

year
2011    3142965
2012    4859704
2013    4328347
2014    3477817
2015    3908496
2016    5553748
Name: word_count, dtype: int64

In [48]:
# Compare number of words between white supremacist dataset forum data and Reddit data
selected_ws = ws_data.loc[(ws_data.timestamp.dt.year.isin(data.year.unique())) & (ws_data.domain == 'forum'),]
selected_ws['word_count'] = selected_ws.text.str.split().str.len()
ws_wc = selected_ws.groupby(ws_data.timestamp.dt.year).word_count.sum()
ws_wc

comparison = pd.concat([ws_wc, reddit_wc], axis=1, keys=['ws_word_count', 'reddit_word_count'])
comparison['reddit_percentage'] = comparison['reddit_word_count']/comparison['ws_word_count']
comparison

/tmp/ipykernel_14819/2852154089.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_ws['word_count'] = selected_ws.text.str.split().str.len()


,ws_word_count,reddit_word_count,reddit_percentage
2011.0,6446972,3142965,0.487510
2012.0,8424657,4859704,0.576843
2013.0,6869854,4328347,0.630049
2014.0,6054223,3477817,0.574445
2015.0,5538318,3908496,0.705719
2016.0,6191536,5553748,0.896990


In [33]:
data.word_count.mean()

55.83427928031435

In [32]:
selected_ws['word_count'].mean()

96.80731973570813

In [21]:
selected_ws['source'].value_counts()

stormfront    49647
ironmarch     37378
Name: source, dtype: int64

In [22]:
selected_ws.groupby('source').word_count.mean()

source
ironmarch      86.517042
stormfront    104.554616
Name: word_count, dtype: float64

In [25]:
# Sample comments vs submissions
from IPython.display import display
pd.set_option('display.max_colwidth', None)

for post_type in ['comment', 'submission']:
    print(post_type)
    display(data.query('post_type == @post_type')[['text']].sample(20))

comment


,text
55771,The same can be said for fascist regimes.
13284,Ignorance.
20569,"To be honest, I'm not particularly interesting in debating the interpretation of the passage. All I'm trying to tell you is that you need to provide something more than some lawyer's interpretation of the passage and your opinion in the matter in order to claim that other people are misinterpreting the passage since they are not evidences or facts. Or else you are just making a baseless claim and it would not be surprising that people don't agree with you."
48625,Wrong about what? (I covered a lot of ground on the last post)
8311,"4 easier states, AK, AZ, Montana I believe, and Vermont, don't require a ccw. But I believe all still have one so you can carry to another state that recognizes said ccw."
11618,"Oh wow, this account has to be the most impressive Markov text generator I've ever seen. Incredible."
40066,"This is a really nice post. See, its helpful, kind, and offers alternatives. Its an example on how not to be a dick, unlike your first post."
2905,"I just have to say that I went to a very bike-friendly university and I worked at a bike shop for a few months (not claiming to be an expert) and I've never been in as bike-friendly a place as Manhattan. \n\nWalking was amazing too, much safer and friendlier than crossing the six-lane roads near where I grew up."
46724,none of which have ever won an election
55711,"To be fair the GTA series of games (which I played and liked) do tend to have a pretty jacked up and sociopathic narrative. While they don't focus on terrorism and there has always a veneer of good vs evil and redemption, the redemptions comes via morally ambiguous mass killings. I'm not arguing that GTA is breeding serial killers or anything like it, just that the subject matter of these games is as perverse and anti social as the subject matter on Fox News."


submission


,text
42918,Gen. David Petraeus arrived at a private lawn party in 2010 with a 28-car motorcade.
1531,Conservative Columnist Charles Krauthammer: Obama Caused GOP Civil War
2574,Letter Allegedly Penned by Chinese Labor Camp Prisoner Found Inside Box of Halloween Decorations from Kmart
18216,"George Will: ""Quite Literally, Opposition To Gay Marriage Is Dying"""
6388,"NRA advocates for armed guards in every school. In fact, Columbine High had an armed guard. He was out monitoring the Smoker's Corner while the shooters did their work inside."
14886,Petition to make illegal for news to lie to their audience in the US
49111,"Romney ""shellshocked"" by loss"
39349,"War: Israel Launches Gaza Invasion, Assassinates Hamas Military Wing Chief"
24511,"Gay Marriage Should be Addressed by SCOTUS, and Championed by Republicans"
42579,UPS ends charitable giving to boy scouts of America over Anti-gay discrimination


# Discord data (chat) 

In [28]:
# Compare with white supremacist dataset
# Load white supremacist dataset to count posts over time
import datetime

path = '../tmp/white_supremacist_corpus.pkl'
ws_data = pd.read_pickle(path)

# Select forum data, Group by year
chat_yearly = ws_data.query('domain=="chat"').groupby(by=ws_data.timestamp.dt.year)['text'].count()
chat_lookup = pd.DataFrame(chat_yearly)
chat_lookup['begin'] = pd.to_datetime(chat_yearly.index.astype(int).astype(str), format='%Y')
chat_lookup['end'] = chat_lookup.begin.shift(-1, fill_value = datetime.datetime(2019,1,1))
chat_lookup.index.name = 'year'
chat_lookup.index = chat_lookup.index.astype(int)
chat_lookup.rename(columns={'text': 'post_count'}, inplace=True)
chat_lookup

,post_count,begin,end
year,,,
2017,33790,2017-01-01,2018-01-01
2018,12340,2018-01-01,2019-01-01


In [ ]:
# Load random Discord data